# `Case study on Credit Risk`


## `Context: `
Credit risk is nothing but the default in payment of any loan by the borrower. In Banking sector this is an important factor to 
be considered before approving the loan of an applicant.Dream Housing Finance company deals in all home loans. They have presence across all urban, semi urban and rural areas. Customer first apply for home loan after that company validates the customer eligibility for loan.


## `Objective:`
Company wants to automate the loan eligibility process (real time) based on customer detail provided while filling online application form. These details are Gender, Marital Status, Education, Number of Dependents, Income, Loan Amount, Credit History and others. To automate this process, they have given a problem to identify the customers segments, those are eligible for loan amount so that they can specifically target these customers. Here they have provided a partial data set.

## `Attributes Information:`

* Variable:	      Description
* Loan_ID:	          Unique Loan ID
* Gender:	          Male/ Female
* Married:	          Applicant married (Y/N)
* Dependents:	      Number of dependents
* Education:	      Applicant Education (Graduate/ Under Graduate)
* Self_Employed:	  Self employed (Y/N)
* ApplicantIncome:	  Applicant income
* CoapplicantIncome: Coapplicant income
* LoanAmount:	      Loan amount in thousands
* Loan_Amount_Term:  Term of loan in months
* Credit_History:	  credit history meets guidelines
* Property_Area:	  Urban/ Semi Urban/ Rural
* Loan_Status:	      Loan approved (Y/N)



----------------------

## `Index`

----------------------
- <a href = #link1>Import Libraries and Load Dataset </a>


- <a href = #link2>Univariate Analysis</a> 


- <a href = #link3>Null Values Treatment</a>


- <a href = #link4>Bivariate Analysis</a> 


- <a href = #link6>Model building using Logistic Regression from Sklearn</a>


- <a href = #link7>Improving Model Performance via Checking Parameters of Logistic Regression</a>


- <a href = #link8>Business Insights</a>


- <a href = #link5>Bonus Content</a>

# `Let's start coding!`

## <a id = "link1"></a> Import Libraries and Load Dataset

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression

# importing ploting libraries
import matplotlib.pyplot as plt
# To enable plotting graphs in Jupyter notebook
%matplotlib inline

#importing seaborn for statistical plots
import seaborn as sns

#Let us break the X and y dataframes into training set and test set. For this we will use
#Sklearn package's data splitting function which is based on random function

from sklearn.model_selection import train_test_split

import numpy as np
from scipy import stats

# calculate accuracy measures and confusion matrix
from sklearn import metrics

In [ ]:
df = pd.read_csv('CreditRisk.csv')
df.head()

In [ ]:
df.shape

**Think about it:**

- What do you interpret by looking at the data in the first 5 rows?
- Do you see that each column is so different from the other. 
    - Numbers of different magnitude plus many columns with categorical values
- There are a lot of columns with categorical data, how to convert this text to numerical value?
    - Gender, Married, Education, Self_Employed

## <a id = "link2"></a> Univariate Analysis

In [ ]:
df.info()

**Think about it:**

- What do you interpret from the different Dtype (data-type) shown here:
     - float64
     - int64
     - object

     How will we deal with the object data-type?

- We know that our dataset has 614 rows of data.
- By looking at the above output of data.info(), we see that some columns are having less number of non-null values.
    - What does this mean?
        - It means some values are NULL (missing)

    How to deal with NULL (missing) values here?

In [ ]:
#Lets analysze the distribution of the various attribute
df.describe().transpose()

**Insights:** `CoapplicantIncome and LoanAmount has 0's that can happen so no need to fill these values`

In [ ]:
df.nunique() # Number of unique values in a column

**Insights:**
- `Loan_Amount_Term is categorical as it has only 10 unique values`
- `Loan_ID column has all unique values and it not a continous column so it has 614 categories one for each row and that would not provide any info to the algorithm`

In [ ]:
cr_df = df.drop('Loan_ID', axis =1 ) # dropping this column as it will be 1-1 mapping anyways
cr_df.head()

In [ ]:
# distplot for continous columns
for i in ['ApplicantIncome','CoapplicantIncome','LoanAmount']:
    sns.distplot(df[i])
    plt.show()

**Insights:**
- `As you can see all the three are skewed but we won't treat it as bank are supposed to have skewed applicants, If we treat the data here it will increase bias`

In [ ]:
cr_df['Loan_Amount_Term'].value_counts(normalize=True)

# value counts gives us how many times does the value 

**Insights:**
- `Loan_Amount_Term most of the values are 360, and rest categories have a very small percentage. So it won't add much value to create dummies of these columns`

In [ ]:
#The Loan_Amount_Term is highly skewed - so we will delete this column
cr_df.drop(['Loan_Amount_Term'], axis=1, inplace=True)

In [ ]:
#Convert X & Y variable to a categorical variable wherever relevant
cr_df['Loan_Status'] = cr_df['Loan_Status'].astype('category')
cr_df['Credit_History'] = cr_df['Credit_History'].astype('category')

**Think about it:**

- We already know that the Dtype (data-type) of the columns, which of them are object or category type. Try to visualize it using countplot.

In [ ]:
cr_df.dtypes

In [ ]:
# for i in ['Gender','Married','Education','Self_Employed','Credit_History','Property_Area','Loan_Status']:
#     sns.countplot(cr_df[i])
#     plt.show()

In [ ]:
for i in list(cr_df.columns[cr_df.dtypes=='object']):   # checking value counts of all object type columns
    print(cr_df[i].value_counts(normalize=True))
    print()

In [ ]:
#Calculate baseline proportion - ratio of Yes to No to identify data imbalance
prop_Y = cr_df['Loan_Status'].value_counts(normalize=True)
print(prop_Y)

**Insights:**
- `There is a slight imbalance in the data but no need to treat it`

## <a id = "link3"></a> Null Values Treatment

In [ ]:
cr_df.isnull().sum()

**Think about it:**

- How will try to fill these null values.
- Is it posiible to draw some relationship between features to fill null values.

In [ ]:
#filling Null Values

cr_df['Credit_History'].fillna(0,inplace=True)

cr_df['Self_Employed'].fillna('No',inplace=True) #replacing with mode

cr_df['Dependents'].fillna('0',inplace=True) #replacing with mode

**Insights:**
- `Credit_History missing values are filled with 0 assuming that they don't meet credit history guidlines .`

- `Self_Employed missing values are replaced with mode, as maximum people are not self employed`

- `Dependents are also filled with mode, assuming most of the people do not have dependents`

In [ ]:
# Drawing relationship between CoapplicantIncome and Gender

cr_df.groupby('Gender')['CoapplicantIncome'].median()

**Insights:**
- `When CoapplicantIncome is 0 and Gender is Female we should fill it with 0.`

In [ ]:
## Filling null values in gender whose CoapplicantIncome is 0 with female

z = cr_df[cr_df['CoapplicantIncome']==0]

for i in list(z[z['Gender'].isnull()].index):
# this would give row numbers of places where gender is null and have CoapplicantIncome as 0
    cr_df.loc[i,'Gender'] = 'Female'

In [ ]:
cr_df['Gender'].fillna('Male',inplace=True) #replacing remaining values with mode as no other relationship found

In [ ]:
#Drawing relationship between Loan_Status, Gender, Property_Area, Married

df.groupby(['Loan_Status','Gender','Property_Area'])['Married'].value_counts()

In [ ]:
cr_df[cr_df['Married'].isnull()]

**Insights:**
- `When Gender is Male Loan Status is 1 and Propert_Area is Urban then mostly they are married.(mode)`

In [ ]:
## Replacing with mode observed in above mentioned relationship:

cr_df.loc[104,'Married'] = 'Yes'
cr_df.loc[228,'Married'] = 'Yes'
cr_df.loc[435,'Married'] = 'No'

In [ ]:
cr_df.isnull().sum()

## <a id = "link4"></a> Bivariate Analysis

**Think about it:**

- In this case-study, we are going to apply Logistic Regression.
- But even before we apply, let's have a look at the data to see how each attribute is different when loan status is yes or not.
- To check it, we ll have to use groupby and crosstabs.

In [ ]:
# Let us look at the target column which is 'Loan_Status' to understand how the data is distributed amongst the various values
cr_df.groupby(["Loan_Status"]).mean()

In [ ]:
cr_df.groupby(["Loan_Status"]).median()

**Insights:**
- `No major difference in case of ApplicantIncome and LoanAmount values w.r.t. target variable`

- `But CoapplicantIncome is higher for people who take loan`

**Think about it**

- Please try to look at the outputs of the codes given below and see if there is any relationship or not

In [ ]:
# pd.crosstab(cr_df['Gender'],cr_df['Loan_Status'],normalize='index')

In [ ]:
# pd.crosstab(cr_df['Married'],cr_df['Loan_Status'],normalize='index')

In [ ]:
# pd.crosstab(cr_df['Education'],cr_df['Loan_Status'],normalize='index')

In [ ]:
pd.crosstab(cr_df['Self_Employed'],cr_df['Loan_Status'],normalize='index')

**Insights:**
- `No difference due to self employed status Threfore dropping this column`

In [ ]:
cr_df.drop('Self_Employed',axis=1,inplace=True)

In [ ]:
pd.crosstab(cr_df['Property_Area'],cr_df['Loan_Status'],normalize='columns')

**Insights:**
- `Semi Urban people are taking more loans`

In [ ]:
pd.crosstab(cr_df['Dependents'],cr_df['Loan_Status'],normalize='index')

**Insights:**
- `Number of dependents have no relationship with Loan_status therfore Dependents is a nominal categorical variable`

## <a id = "link6"></a> Model building using Logistic Regression from Sklearn 

We will use the sklearn library to build the model and make predictions

In [ ]:
## Define X and Y variables
X = cr_df.drop('Loan_Status', axis=1)
Y = cr_df[['Loan_Status']]

In [ ]:
#Convert categorical vriables to dummy variables
X = pd.get_dummies(X, drop_first=True)

**Think about it**

- How does drop first help and why we do it
  - It reduces the number of columns and hence increases the computational time.

In [ ]:
##Split into training and test set
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30,random_state=7)

**Think about it**

- Why do we use random State
  - Sklearn randomly shuffles the data so we get a slightly different answer. If we fix the random state and re-run the code then all the scores will stay the same for that random state.
  - If we do not use random state our metrics will slightly differ in every run.

In [ ]:
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score, roc_auc_score,accuracy_score
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(random_state=7)
logreg.fit(X_train, y_train)                    # fit the model on train data

In [ ]:
y_predict = logreg.predict(X_test)              # Predicting the target variable on test data

In [ ]:
# Observe the predicted and observed classes in a dataframe.

z = X_test.copy()
z['Observed Loan Status'] = y_test
z['Predicted Loan Status'] = y_predict
z.head()

In [ ]:
## function to get confusion matrix in a proper format
def draw_cm( actual, predicted ):
    cm = confusion_matrix( actual, predicted)
    sns.heatmap(cm, annot=True,  fmt='.2f', xticklabels = [0,1] , yticklabels = [0,1] )
    plt.ylabel('Observed')
    plt.xlabel('Predicted')
    plt.show()

In [ ]:
print("Trainig accuracy",logreg.score(X_train,y_train))  
print()
print("Testing accuracy",logreg.score(X_test, y_test))
print()
print('Confusion Matrix')
print(draw_cm(y_test,y_predict))
print()
print("Recall:",recall_score(y_test,y_predict))
print()
print("Precision:",precision_score(y_test,y_predict))
print()
print("F1 Score:",f1_score(y_test,y_predict))
print()
print("Roc Auc Score:",roc_auc_score(y_test,y_predict))

In [ ]:
# !pip install yellowbrick

# Additional

#AUC ROC curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

## <a id = "link7"></a> Improving Model Performance via Checking Parameters of Logistic Regression

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

`This is an introduction to let you know that all the algorithms have parameters aand changing those parameters affects the model performance. How to regularize parameter is covered in FMST module.`

In [ ]:
# Checking Parameters of logistic regression
logreg.get_params()

#If we dont specify the parameters in the model it takes default value

**Think about it**

- How to improve model performance
    - By checking the parameters and try changing it and then verify using model metrics whether performance increased or not.    

In [ ]:
# Running a loop to check different values of 'solver'
# all solver can be used with l2, only 'liblinear' and 'saga' works with both 'l1' and 'l2'

train_score=[]
test_score=[]
solver = ['newton-cg','lbfgs','liblinear','sag','saga']
for i in solver:
    model = LogisticRegression(random_state=42,penalty='l2', C = 0.75,solver=i)  # changing values of solver
    model.fit(X_train, y_train) 
    y_predict = model.predict(X_test)     
    train_score.append(round(model.score(X_train, y_train),3))
    test_score.append(round(model.score(X_test, y_test),3))
    
print(solver)
print()
print(train_score)
print()
print(test_score)

In [ ]:
train_score=[]
test_score=[]
solver = ['liblinear','saga']   # changing values of solver which works with 'l1'
for i in solver:
    model = LogisticRegression(random_state=42,penalty='l1', C = 0.75,solver=i)  #changed penalty to 'l1'
    model.fit(X_train, y_train) 
    y_predict = model.predict(X_test)     
    train_score.append(round(model.score(X_train, y_train),3))
    test_score.append(round(model.score(X_test, y_test),3))
    
print(solver)
print()
print(train_score)
print()
print(test_score)

**Insights:**
- `Highest accuracy is same 'l1' with 'liblinear' and 'l2' with 'newton-cg'`

choose any one

In [ ]:
model = LogisticRegression(random_state=42,penalty='l1',solver='liblinear',class_weight='balanced') # changing class weight to balanced

model.fit(X_train, y_train) 

y_predict = model.predict(X_test)     

print("Trainig accuracy",model.score(X_train,y_train))  
print()
print("Testing accuracy",model.score(X_test, y_test))

**Insights:**
- `Testing accuracy increased and model is not overfit anymore so adding class weight from the model`

In [ ]:
# Running a loop to check different values of 'C'

train_score=[]                                 
test_score=[]
C = [0.01,0.1,0.25,0.5,0.75,1]
for i in C:
    model = LogisticRegression(random_state=42,penalty='l1', solver='liblinear',class_weight='balanced', C=i)  # changing values of C
    model.fit(X_train, y_train) 
    y_predict = model.predict(X_test)     
    train_score.append(round(model.score(X_train,y_train),3)) # appending training accuracy in a blank list for every run of the loop
    test_score.append(round(model.score(X_test, y_test),3))   # appending testing accuracy in a blank list for every run of the loop
    
print(C)
print()
print(train_score)
print()
print(test_score)

**Insights:**
- `Best testing accuracy is obtained for C=0.5, which is default`

In [ ]:
#Therefore final model is
model = LogisticRegression(random_state=42,penalty='l1', solver='liblinear', class_weight='balanced',C=0.5) 
model.fit(X_train, y_train)
y_predict = model.predict(X_test)
print("Trainig accuracy",model.score(X_train,y_train))  
print()
print("Testing accuracy",model.score(X_test, y_test))
print()
print('Confusion Matrix')
print(draw_cm(y_test,y_predict))
print()
print("Recall:",recall_score(y_test,y_predict))
print()
print("Precision:",precision_score(y_test,y_predict))
print()
print("F1 Score:",f1_score(y_test,y_predict))
print()
print("Roc Auc Score:",roc_auc_score(y_test,y_predict))

In [ ]:
# !pip install yellowbrick

In [ ]:
# Additional

from yellowbrick.classifier import ClassificationReport, ROCAUC
# Visualize model performance with yellowbrick library
viz = ClassificationReport(model)
viz.fit(X_train, y_train)
viz.score(X_test, y_test)
viz.show()

roc = ROCAUC(model)
roc.fit(X_train, y_train)
roc.score(X_test, y_test)
roc.show()

## <a id = "link8"></a>Business Insights 

`Confusion matrix means`

*True Positive (observed=1,predicted=1):*

Predicted that home loan will be granted and the customer was eligibile for loan

*False Positive (observed=0,predicted=1):*

Predicted that home loan will be granted and the customer was not eligibile for loan

*True Negative (observed=0,predicted=0):*

Predicted that home loan will not be granted and the customer was not eligibile for loan

*False Negative (observed=1,predicted=0):*

Predicted that home loan will not be granted and the customer was eligibile for loan

Here the bank wants to give loan to the people who are eligible for the home loan i.e. **less number of False Positive**, if FP is high bank would lose money. So that the bank doesn't lose money on the people who are not eligible for the loan. Hence **Precision is the important metric**.

In case of False negative bank will lose few customers but that okay because the bank would want to retain money more than customers who are not eligible for loan.

After achieving the desired accuracy we can deploy the model for practical use. As in the bank now can predict who is eligible for home loan. They can use the model for upcoming customers.

## <a id = "link5"></a>Bonus Content

Model building using Statsmodel.api

In [ ]:
#Build the logistic regression model
import statsmodels.api as sm

logit = sm.Logit(y_train, sm.add_constant(X_train))
lg = logit.fit()

In [ ]:
#Summary of logistic regression
from scipy import stats
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)
print(lg.summary())

### Interpretation of Pseudo R^2

A pseudo R^2 of 17% indicates that 17% of the uncertainty of the intercept only model is explained by the full model

#### Calculate the odds ratio from the coef using the formula odds ratio=exp(coef)

#### Calculate the probability from the odds ratio using the formula probability = odds / (1+odds)

In [ ]:
#Calculate Odds Ratio, probability
##create a data frame to collate Odds ratio, probability and p-value of the coef
lgcoef = pd.DataFrame(lg.params, columns=['coef'])
lgcoef.loc[:, "Odds_ratio"] = np.exp(lgcoef.coef)
lgcoef['probability'] = lgcoef['Odds_ratio']/(1+lgcoef['Odds_ratio'])
lgcoef['pval']=lg.pvalues
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
# FIlter by significant p-value (pval <0.1) and sort descending by Odds ratio
lgcoef = lgcoef.sort_values(by="Odds_ratio", ascending=False)
pval_filter = lgcoef['pval']<=0.1
lgcoef[pval_filter]

**Insights:**
- `Customers with credit history 1 have a 89% probability of defaulting the loan`

- `Customers in semiurban areas have an odds of 2.50 times to default`

## Appendix:

- **warnings.filterwarnings("ignore")** : Never print matching warnings.

- **Pandas** : Pandas is an open source library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language.

- **Numpy** : The fundamental package for scientific computing with Python.

- **Matplotlib** : Matplotlib is a comprehensive library for creating static, animated, and interactive visualizations in Python.

- **Seaborn** : Seaborn is a Python data visualization library based on matplotlib. It provides a high-level interface for drawing attractive and informative statistical graphics.

- **pairplot** : Plot pairwise relationships in a dataset.

- **boxplot** : Draw a box plot to show distributions with respect to categories.

- **distplot** : Flexibly plot a univariate distribution of observations.

- **pandas.DataFrame.corr** : Compute pairwise correlation of columns, excluding NA/null values.

- **sklearn.linear_model.LogisticRegression** : Logistic Regression (aka logit, MaxEnt) classifier.

- **scipy.stats** : This module contains a large number of probability distributions as well as a growing library of statistical functions.

- **statsmodels** : It is a Python module that provides classes and functions for the estimation of many different statistical models, as well as for conducting statistical tests, and statistical data exploration.
    - An extensive list of result statistics are available for each estimator.
    - The results are tested against existing statistical packages to ensure that they are correct.
    - The online documentation is hosted at [statsmodels.org](https://www.statsmodels.org/stable/index.html).

- **yellowbrick** : Yellowbrick extends the Scikit-Learn API to make model selection and hyperparameter tuning easier. Under the hood, it  uses Matplotlib